## WebScrapping - Valor Econômico

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
from selenium import webdriver
import pandas as pd
import datetime
from datetime import date
from datetime import datetime
import urllib.request
from urllib.parse import urlparse



# Novas
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
%%time
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
navegador = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

### Parte 1: Extração das URLs Diárias de Publicações do Valor Econômico

Este segmento do processo envolve a análise do 'sitemap.xml' do Valor Econômico para identificar e coletar as URLs que correspondem às edições diárias. Cada uma dessas URLs contém os links para os artigos publicados em um dia específico.

In [ ]:
sitemap = 'https://valor.globo.com/sitemap/valor/sitemap.xml'
navegador.get(sitemap)
sleep(1)
site = BeautifulSoup(navegador.page_source, 'xml')

links_sitemap = []

links = site.findAll('loc')

for link in links:
    
    url = link.text
    
    links_sitemap.append(url)

### Parte 2: Compilação das Notícias Diárias do Valor Econômico

Nesta etapa, consolidamos todas as notícias acessando as URLs das edições diárias previamente identificadas, coletando assim o conteúdo publicado para cada dia.

In [ ]:
%%time

links_noticias = []

for i in tqdm(range(0,len(links_sitemap))):
    navegador.get(links_sitemap[i])
    sleep(1)
    site = BeautifulSoup(navegador.page_source, 'xml')
    
    noticias = site.findAll('url')
    
    for noticia in noticias:
        
        html = noticia.find('loc').text
        links_noticias.append(html)

In [ ]:
# Número de Notícias
len(links_noticias)

In [ ]:
# Transformando em DataFrame
df_links = pd.DataFrame(links_noticias)
df_links.to_csv('LN_Valor.csv', index=False)

### Parte 3: Login Valor

In [ ]:
navegador.get('https://login.globo.com/login/6908/connect-confirm?url=https%3A%2F%2Fid.globo.com%2Fauth%2Frealms%2Fglobo.com%2Flogin-actions%2Fauthenticate%3Fsession_code%3Dc9e54WgBYYO_tXm-neZ-sC_uKKfpxgaJz0XC2Xer_j0%26execution%3Db5dd88dc-447e-468f-945e-e7c7de4883b7%26client_id%3Dvalor%2540globoid-connect%26tab_id%3D9DJDno0akOQ%26request-context%3DcluCi4&error=&request-context=cluCi4')
sleep(1)

email = 'seu-email'
senha = 'sua senha'


navegador.find_element_by_name("login").send_keys(email)
sleep(1)
navegador.find_element_by_name("password").send_keys(senha)
sleep(1)


navegador.find_element_by_css_selector('#login-form > div.actions > button').click()

### Parte 4: Texto, Título e Data 

In [ ]:
df_links = pd.read_csv('LN_Valor.csv')
news_links = df_links['0'].tolist()

In [ ]:
# Tirando os links que possuem o termo 'ao-vivo', já que se tratam de coberturas.
news_links = [item for item in news_links if 'ao-vivo' not in item]
# Tirando os links patrocinados
news_links = [item for item in news_links if 'patrocinado' not in item]
# Retirando os Stories
news_links = [item for item in news_links if 'stories' not in item]
len(news_links) # Novo número de Notícias

In [ ]:
# Retirar páginas relacionadas a 'eu-e'
ok = [item for item in news_links if 'eu-e' not in item]
# Retirar páginas relacionadas a carreiras
ok1 = [item for item in ok if 'carreira' not in item]
# Retirar páginas relacionadas a 'especiais'
ok3 = [item for item in ok1 if 'especiais' not in item]
len(ok3) # Novo número de Notícias

# Extração dos Dados das Publicações

Ajuste a variável j de acordo com a capacidade de memória e processamento do seu computador. Atualmente, o script está configurado para extrair dados de 10.000 notícias por sessão, mas este valor pode ser modificado conforme necessário. Dependendo do volume de dados, pode ser preciso dividir o processo de raspagem de dados em várias etapas. Além disso, para minimizar o risco de bloqueio por parte do site do Valor Econômico devido a atividades de raspagem intensiva, foi implementado um intervalo de espera (sleep) de 0.6 segundos antes de cada coleta de dados após acessar a página. Pórem, esse valor ainda pode ser insuficiente em alguns momentos. 

Caso a coleta de dados não seja bem-sucedida para algumas notícias durante a sessão de extração de 10.000 artigos, os respectivos links serão armazenados em um dataframe(**Erro_p{j}.csv**). Este dataframe estará disponível para que possamos retomar e concluir a raspagem de dados dos artigos que inicialmente encontraram erros.



In [ ]:
links_com_erro=list()
erros = list()

for j in range(0,1):
    news_1 = []

    for i in tqdm(range(j*10000,(j+1)*10000)):
        try:
            url = news_links[i]
            navegador.get(url)

            sleep(0.6)
            site = BeautifulSoup(navegador.page_source, 'html.parser')

            titulo = site.find('h1', attrs={'class':'content-head__title'}).text
            data = site.find('time', attrs={'itemprop':'datePublished'}).text.replace('h',':')
            data1 = datetime.strptime(data,' %d/%m/%Y %H:%M ')

            article_text = ''
            article = site.find('article', attrs={'itemprop':'articleBody'}).findAll('div', attrs={'class':'mc-column content-text active-extra-styles'})
            if article:
                for element in article:
                    article_text += ''.join(element.text)
            else:
                article_text = site.find('article', attrs={'itemprop':'articleBody'}).text
                
            news_1.append([data1, titulo, article_text, url])
            
        except Exception as e:
            print(e)
            links_com_erro.append(url)
                    

    valor_p1 = pd.DataFrame(news_1, columns=['Data', 'Título', 'Texto', 'Url'])
    valor_p1.to_csv(f'Valor_p{j}.csv', index=False)
    erro_p1 = pd.DataFrame(links_com_erro)
    erro_p1.to_csv(f'Erro_p{j}.csv', index=False)